In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 0)

In [167]:
# LOAD 2023 PROPERTY SALE DATA
df_sale_2023_raw = pd.read_csv('jabodetabek_house_price.csv')

In [166]:
# LOAD 2025 PROPERTY SALE & RENT DATA

df_sale_raw = pd.read_csv('rumah123_sale_properties.csv')
df_rent_raw = pd.read_csv('rumah123_rent_properties.csv')

In [ ]:
# CLEANING PROCESS:
'''
- remove building_age feature
- re-order columns
- rename + re-order sale_2023 data
- check the data types of each column
    - if not numeric, convert to numeric (e.g. bathrooms, bedrooms)
- convert price_in_rp, electricity, land_size, building_size  to numeric

- check city unique values. Only keep Jabodetabek cities (Jakarta, Bogor, Depok, Tangerang, Bekasi). -> clean (e.g. Bandung writen as Badung)

- identify duplicated columns
- Handling null/na values: replace/remove

- feature understanding (univariate analysis): histogram, KDE, boxplot (identifying outliers).
    checking and removing outliers.
'''

# DATA PREPARATION (XGBoost)
'''
- data normalization (e.g. price and building area scale are far apart)

- One Hot Encoding (categorical -> numerical for XGBoost)
'''

### DATA CLEANING

In [316]:
# Remove building_age feature
df_sale = df_sale_raw.drop(columns=['building_age'], errors='ignore')
df_rent = df_rent_raw.drop(columns=['building_age'], errors='ignore')

# re-order columns
df_sale = df_sale[['url', 'title', 'price_in_rp', 'location', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2','certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]

df_rent = df_rent[['url', 'title', 'price_in_rp', 'location', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2','certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]
df_rent.rename(columns={'price_in_rp': 'rent_price_per_year'}, inplace=True)

In [317]:
# rename + re-order sale_2023 data
df_sale_2023 = df_sale_2023_raw.rename(columns={
    'url': 'url', 
    'price_in_rp': 'price_in_rp', 
    'title': 'title', 
    'address': 'location', 
    'district': 'district', 
    'city': 'city', 
    # 'lat',
    # 'long', 
    'facilities': 'description', 
    # 'property_type', 
    # 'ads_id', 
    'bedrooms': 'bedrooms',
    'bathrooms': 'bathrooms', 
    'land_size_m2': 'land_size_m2', 
    'building_size_m2': 'building_size_m2', 
    # 'carports'
    'certificate': 'certificate', 
    'electricity': 'electricity_power', 
    # 'maid_bedrooms', 
    # 'maid_bathrooms',
    'floors': 'floor_count', 
    # 'building_age', 
    # 'year_built', 
    'property_condition': 'property_condition',
    # 'building_orientation', 
    # 'garages', 
    # 'furnishing'    
}).copy()

df_sale_2023 = df_sale_2023[['url', 'title', 'price_in_rp', 'location', 'district', 'city' , 'bedrooms', 'bathrooms', 'land_size_m2', 'building_size_m2', 'certificate', 'electricity_power', 'property_condition', 'floor_count', 'description']]

In [318]:
# check the data types of each column prior to conversion
print(df_sale.dtypes)
print(df_rent.dtypes)

url                   object 
title                 object 
price_in_rp           object 
location              object 
district              object 
city                  object 
bedrooms              float64
bathrooms             float64
land_size_m2          object 
building_size_m2      object 
certificate           object 
electricity_power     object 
property_condition    object 
floor_count           float64
description           object 
dtype: object
url                    object 
title                  object 
rent_price_per_year    object 
location               object 
district               object 
city                   object 
bedrooms               float64
bathrooms              float64
land_size_m2           object 
building_size_m2       object 
certificate            object 
electricity_power      object 
property_condition     object 
floor_count            float64
description            object 
dtype: object


In [319]:
# NUMERIC DATA TYPE CONVERSION (2025 DATA)

# --- price_in_rp ---
for i, val in enumerate(df_sale['price_in_rp']):
    if isinstance(val, str):
        val = val.replace('Rp', '').replace(',', '.')
        if 'Miliar' in val:
            val = val.replace('Miliar', '').strip()
            val = float(val) * 1_000_000_000
        elif 'Juta' in val:
            val = val.replace('Juta', '').strip()
            val = float(val) * 1_000_000
        df_sale.at[i, 'price_in_rp'] = val
    # print(i, val)

for i, val in enumerate(df_rent['rent_price_per_year']):
    if isinstance(val, str):
        val = val.replace('Rp', '').replace(',', '.')
        if '/bulan' in val or 'per bulan' in val:
            val = val.replace('/bulan', '').replace('per bulan', '').strip()
            period = 12
        elif '/tahun' in val or 'per tahun' in val:
            val = val.replace('/tahun', '').replace('per tahun', '').strip()
            period = 1
        elif '/hari' in val or 'per hari' in val:
            val = val.replace('/hari', '').replace('per hari', '').strip()
            period = 365
            
        if 'Miliar' in val:
            val = val.replace('Miliar', '').strip()
            val = float(val) * 1_000_000_000 * period
        elif 'Juta' in val:
            val = val.replace('Juta', '').strip()
            val = float(val) * 1_000_000 * period
        df_rent.at[i, 'rent_price_per_year'] = float(val)
    # print(i, val)


# --- land_size_m2 ---
if df_sale['land_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_sale['land_size_m2'] = df_sale['land_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)
if df_rent['land_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_rent['land_size_m2'] = df_rent['land_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)

# --- building_size_m2 ---
if df_sale['building_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_sale['building_size_m2'] = df_sale['building_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)
if df_rent['building_size_m2'].dtype == 'object':
    # Remove 'm²' and convert to float
    df_rent['building_size_m2'] = df_rent['building_size_m2'].str.replace('m²', '').str.replace(',', '.').astype(float)

# --- electricity_power ---
if df_sale['electricity_power'].dtype == 'object':
    # Remove 'Watt' and convert to float
    df_sale['electricity_power'] = df_sale['electricity_power'].str.replace('Lainnya', '0').str.replace('Watt', '').str.replace(',', '.').astype(float)
if df_rent['electricity_power'].dtype == 'object':
    # Remove 'Watt' and convert to float
    df_rent['electricity_power'] = df_rent['electricity_power'].str.replace('Lainnya', '0').str.replace('Watt', '').str.replace(',', '.').astype(float)

# --- property_condition ---
'''
Baru = 5
Bagus = 4
Sudah Renovasi = 3
Renovasi Minimum = 2
Butuh Renovasi = 1
Renovasi Total = 0
'''
property_condition_mapping = {
    'Baru': 5,
    'Bagus': 4,
    'Sudah Renovasi': 3,
    'Renovasi Minimum': 2,
    'Butuh Renovasi': 1,
    'Renovasi Total': 0
}
if df_sale['property_condition'].dtype == 'object':
    df_sale['property_condition'] = df_sale['property_condition'].map(property_condition_mapping)
if df_rent['property_condition'].dtype == 'object':
    df_rent['property_condition'] = df_rent['property_condition'].map(property_condition_mapping)


# CATEGORICAL DATA CLEANING
# standardize certificate values
df_sale['certificate'] = df_sale['certificate'].replace(
    [
        'PPJB', 'Hak Sewa'
    ], 
    'Lainnya')
df_rent['certificate'] = df_rent['certificate'].replace(
    [
        'PPJB', 'Hak Sewa'
    ],
    'Lainnya')

In [320]:
# check the data types of each column after conversion
print(df_sale.dtypes, end='\n\n')
print(df_rent.dtypes)

url                   object 
title                 object 
price_in_rp           object 
location              object 
district              object 
city                  object 
bedrooms              float64
bathrooms             float64
land_size_m2          float64
building_size_m2      float64
certificate           object 
electricity_power     float64
property_condition    float64
floor_count           float64
description           object 
dtype: object

url                    object 
title                  object 
rent_price_per_year    object 
location               object 
district               object 
city                   object 
bedrooms               float64
bathrooms              float64
land_size_m2           float64
building_size_m2       float64
certificate            object 
electricity_power      float64
property_condition     float64
floor_count            float64
description            object 
dtype: object


In [321]:
# NUMERIC DATA TYPE CONVERSION (2023 HOUSE SALES DATA)

# --- electricity_power ---
'''
- convert mah to watt. 1 Watt = 1 mAh (this is an assumption because somehow the range of electricity between watt in 2025 data and mah in 2023 data is similar)
'''
if df_sale_2023['electricity_power'].dtype == 'object':
    # Replace 'lainnya' with 0, remove 'mah', replace ',' with '.' and convert to float
    df_sale_2023['electricity_power'] = df_sale_2023['electricity_power'].str.replace('lainnya', '0').str.replace('mah', '').str.replace(',', '.').astype(float)

# --- property condition mapping ---
property_condition_mapping = {
    'baru': 5,
    'bagus sekali': 5,
    'bagus': 4,
    'sudah renovasi': 3,
    'semi furnished': 3,
    'butuh renovasi': 2,
    'unfurnished': 2
}
if df_sale_2023['property_condition'].dtype == 'object':
    df_sale_2023['property_condition'] = df_sale_2023['property_condition'].map(property_condition_mapping)



# CATEGORICAL DATA CLEANING
# standardize certificate values
df_sale_2023['certificate'] = df_sale_2023['certificate'].replace(
    [
        'shm - sertifikat hak milik', 'hgb - hak guna bangunan', 'lainnya (ppjb,girik,adat,dll)', 'hp - hak pakai'
    ], 
    [
        'SHM', 'HGB', 'Lainnya', 'Hak Pakai'
    ])

# trim spaces and whitespaces for categorical features
df_sale_2023['location'] = df_sale['location'].str.strip()
df_sale_2023['district'] = df_sale['district'].str.strip()
df_sale_2023['city'] = df_sale_2023['city'].str.strip()

In [322]:
# check city unique values. Only keep Jabodetabek cities (Jakarta, Bogor, Depok, Tangerang, Bekasi).
df_sale.query("city == ['Bekasi', 'Bogor', 'Depok', 'Jakarta Barat', 'Jakarta Selatan', 'Jakarta Utara', 'Jakarta Timur', 'Jakarta Pusat', 'Tangerang']", inplace=True)
df_rent.query("city == ['Bekasi', 'Bogor', 'Depok', 'Jakarta Barat', 'Jakarta Selatan', 'Jakarta Utara', 'Jakarta Timur', 'Jakarta Pusat', 'Tangerang']", inplace=True)

In [362]:
# DUPLICATE HANDLING
'''
there's no duplicates in the dataset: df_sale, df_rent, df_sale_2023
'''

# NULL VALUES HANDLING
    # if we don't have time, just dropna. But if we do, do this:
'''
df_sale:
- check ourself from the image or description: bedrooms, bathrooms, floor_count, property_condition

df_rent:
- check ourself from the image or description: bedrooms, bathrooms, land_size_m2, building_size_m2
- fill null: floor_count = 1 (check first)
- drop null: property_condition

df_sale_2023:
- drop null: location, land_size_m2, building_size_m2, certificate, property_condition, floor_count
'''

# df_sale:
df_sale.dropna(subset=df_sale.columns[df_sale.isnull().any()].tolist(), inplace=True)

# df_rent:
df_rent.dropna(subset=df_rent.columns[df_rent.isnull().any()].tolist(), inplace=True)

# df_sale_2023:
df_sale_2023.dropna(subset=df_sale_2023.columns[df_sale_2023.isnull().any()].tolist(), inplace=True)

### FEATURE UNDERSTANDING

In [ ]:
# cleaned dataframes
df_sale, df_rent, df_sale_2023

(                                                               url  \
 1     https://www.rumah123.com/properti/bekasi/hos38968444/          
 2     https://www.rumah123.com/properti/bogor/hos39351133/           
 3     https://www.rumah123.com/properti/bogor/hos39293477/           
 4     https://www.rumah123.com/properti/depok/hos39296796/           
 5     https://www.rumah123.com/properti/depok/hos39292101/           
 ...                                                    ...           
 2589  https://www.rumah123.com/properti/tangerang/hos39329111/       
 2590  https://www.rumah123.com/properti/tangerang/hos39351912/       
 2592  https://www.rumah123.com/properti/jakarta-utara/hos39345486/   
 2599  https://www.rumah123.com/properti/jakarta-utara/hos39316412/   
 2600  https://www.rumah123.com/properti/jakarta-utara/hos39147082/   
 
                                                                           title  \
 1     Rumah Hadap Utara 13 menit ke Plaza Taman Harapan Baru 

In [ ]:
# check for outliers